In [13]:
import plotly.graph_objects as go
import pandas as pd
import random
import threading

In [14]:
csv_file:str = "migrations.csv"

In [15]:
DRUGBANK_DATABASES:list() = [
#(index, DB name, db xml file, release month, release year)
(0,"DB 3.0","database/3.0/drugbank.xml",1,2011),
(1,"DB 4.1","database/4.1/drugbank.xml",12,2014),
(2,"DB 4.2","database/4.2/drugbank.xml",2,2015),
(3,"DB 4.3","database/4.3/drugbank.xml",2,2016),
(4,"DB 4.5","database/4.5.0/drugbank.xml",4,2016),
(5," DB 5.0","database/5.0.0/drugbank.xml",6,2016),
(6,"DB 5.0.1","database/5.0.1/drugbank.xml",7,2016),
(7,"DB 5.0.2","database/5.0.2/drugbank.xml",10,2016),
(8,"DB 5.0.3","database/5.0.3/drugbank.xml",10,2016),
(9,"DB 5.0.4","database/5.0.4/full_database.xml",1,2017),
(10,"DB 5.0.5","database/5.0.5/full_database.xml",1,2017),
(11,"DB 5.0.6","database/5.0.6/full_database.xml",4,2017),
(12,"DB 5.0.7","database/5.0.7/full_database.xml",7,2017),
(13,"DB 5.0.8","database/5.0.8/full_database.xml",9,2017),
(14,"DB 5.0.9","database/5.0.9/full_database.xml",10,2017),
(15,"DB 5.0.10","database/5.0.10/full_database.xml",11,2017),
(16,"DB 5.0.11","database/5.0.11/full_database.xml",12,2017),
(17,"DB 5.1.0","database/5.1.0/full_database.xml",4,2018),
(18,"DB 5.1.1","database/5.1.1/full_database.xml",7,2018),
(19,"DB 5.1.2","database/5.1.2/full_database.xml",12,2018),
(20,"DB 5.1.3","database/5.1.3/full_database.xml",4,2019),
(21,"DB 5.1.4","database/5.1.4/full_database.xml",7,2019),
(22,"DB 5.1.5","database/5.1.5/full_database.xml",1,2020),
(23,"DB 5.1.6","database/5.1.6/full_database.xml",4,2020),
(24,"DB 5.1.7","database/5.1.7/full_database.xml",7,2020),
(25,"DB 5.1.8","database/5.1.8/full_database.xml",1,2021)
]

In [16]:
#df_first = pd.read_csv(csv_file)
#df=df_first.head()
df = pd.read_csv(csv_file)

In [17]:
entries=list()
for intex,row in df.iterrows():
    for i in range(0,len(DRUGBANK_DATABASES)-1):
        entry={"drug_name":"","source_cluster":0, "destination_cluster":0,"source_db":"", "destination_db":""}
        entry["drug_name"] = row["Drug Name"]
        entry["source_cluster"]=row[DRUGBANK_DATABASES[i][1]]
        entry["source_db"]=DRUGBANK_DATABASES[i][1]
        entry["destination_cluster"]=row[DRUGBANK_DATABASES[i+1][1]]
        entry["destination_db"]=DRUGBANK_DATABASES[i+1][1]
        entries.append(entry)


In [25]:
class DiagramData:

    def __init__(self):
        r = lambda: random.randint(0,255)
        self.mapped_labels= [(item[1],j,"rgba({0}, {1}, {2}, 0.8)".format(r(),r(),r())) for item in DRUGBANK_DATABASES for j in range(1,4)]
        self.source=list()
        self.target=list()
        self.labels=list()
        self.values=list()
        self.colors=list()
        self.lock = threading.Lock()
    
    def parse_data(self):
        self.entries=list()
        for index,row in df.iterrows():
            for i in range(0,len(DRUGBANK_DATABASES)-1):
                entry={"drug_name":"","source_cluster":0, "destination_cluster":0,"source_db":"", 
                       "destination_db":"","source_index":0,"destination_index":0}
                entry["drug_name"] = row["Drug Name"]
                entry["source_cluster"]=row[DRUGBANK_DATABASES[i][1]]
                entry["source_db"]=DRUGBANK_DATABASES[i][1]
                entry["destination_cluster"]=row[DRUGBANK_DATABASES[i+1][1]]
                entry["destination_db"]=DRUGBANK_DATABASES[i+1][1]
                for j in range(0,len(self.mapped_labels)):
                    if self.mapped_labels[j][0]==entry["source_db"] and self.mapped_labels[j][1]==entry["source_cluster"]:
                            entry["source_index"]=j
                    if self.mapped_labels[j][0]==entry["destination_db"] and self.mapped_labels[j][1]==entry["destination_cluster"]:
                            entry["destination_index"]=j
                self.entries.append(entry)
                
    def thread_function(self,name):
        self.values[name]=sum(map(lambda item:item["source_index"]==self.source[name] and item["destination_index"]==self.target[name],self.entries))
        


    def generate_links_threads(self):
        for item in self.entries:
            self.source.append(item["source_index"])
            self.target.append(item["destination_index"])
        self.labels=[item[0]+" C: "+str(item[1]) for item in self.mapped_labels]
        self.color =[item[2] for item in self.mapped_labels]
        for i in range(0,len(self.source)):
            self.values.append(0)
        threads = list()
        
        for i in range(0,len(self.source)):
            print("Thread {0}".format(i))
            x = threading.Thread(target=self.thread_function, args=(i,))
            threads.append(x)
        for thread in threads:
            thread.start()
        for index, thread in enumerate(threads):
            print("Main    : before joining thread {0}.".format(index))
            thread.join()

    def generate_links(self):
        for item in self.entries:
            self.source.append(item["source_index"])
            self.target.append(item["destination_index"])
        self.labels=[item[0]+" C: "+str(item[1]) for item in self.mapped_labels]
        self.color =[item[2] for item in self.mapped_labels]
        for i in range(0,len(self.source)):
            total_entries= sum(map(lambda item:item["source_index"]==self.source[i] and item["destination_index"]==self.target[i],self.entries))
            self.values.append(total_entries)

In [26]:
var=DiagramData()
var.parse_data()
var.generate_links_threads()


Thread 0
Thread 1
Thread 2
Thread 3
Thread 4
Thread 5
Thread 6
Thread 7
Thread 8
Thread 9
Thread 10
Thread 11
Thread 12
Thread 13
Thread 14
Thread 15
Thread 16
Thread 17
Thread 18
Thread 19
Thread 20
Thread 21
Thread 22
Thread 23
Thread 24
Thread 25
Thread 26
Thread 27
Thread 28
Thread 29
Thread 30
Thread 31
Thread 32
Thread 33
Thread 34
Thread 35
Thread 36
Thread 37
Thread 38
Thread 39
Thread 40
Thread 41
Thread 42
Thread 43
Thread 44
Thread 45
Thread 46
Thread 47
Thread 48
Thread 49
Thread 50
Thread 51
Thread 52
Thread 53
Thread 54
Thread 55
Thread 56
Thread 57
Thread 58
Thread 59
Thread 60
Thread 61
Thread 62
Thread 63
Thread 64
Thread 65
Thread 66
Thread 67
Thread 68
Thread 69
Thread 70
Thread 71
Thread 72
Thread 73
Thread 74
Thread 75
Thread 76
Thread 77
Thread 78
Thread 79
Thread 80
Thread 81
Thread 82
Thread 83
Thread 84
Thread 85
Thread 86
Thread 87
Thread 88
Thread 89
Thread 90
Thread 91
Thread 92
Thread 93
Thread 94
Thread 95
Thread 96
Thread 97
Thread 98
Thread 99
Thread 100

Thread 3411
Thread 3412
Thread 3413
Thread 3414
Thread 3415
Thread 3416
Thread 3417
Thread 3418
Thread 3419
Thread 3420
Thread 3421
Thread 3422
Thread 3423
Thread 3424
Thread 3425
Thread 3426
Thread 3427
Thread 3428
Thread 3429
Thread 3430
Thread 3431
Thread 3432
Thread 3433
Thread 3434
Thread 3435
Thread 3436
Thread 3437
Thread 3438
Thread 3439
Thread 3440
Thread 3441
Thread 3442
Thread 3443
Thread 3444
Thread 3445
Thread 3446
Thread 3447
Thread 3448
Thread 3449
Thread 3450
Thread 3451
Thread 3452
Thread 3453
Thread 3454
Thread 3455
Thread 3456
Thread 3457
Thread 3458
Thread 3459
Thread 3460
Thread 3461
Thread 3462
Thread 3463
Thread 3464
Thread 3465
Thread 3466
Thread 3467
Thread 3468
Thread 3469
Thread 3470
Thread 3471
Thread 3472
Thread 3473
Thread 3474
Thread 3475
Thread 3476
Thread 3477
Thread 3478
Thread 3479
Thread 3480
Thread 3481
Thread 3482
Thread 3483
Thread 3484
Thread 3485
Thread 3486
Thread 3487
Thread 3488
Thread 3489
Thread 3490
Thread 3491
Thread 3492
Thread 3493
Thre

Thread 5911
Thread 5912
Thread 5913
Thread 5914
Thread 5915
Thread 5916
Thread 5917
Thread 5918
Thread 5919
Thread 5920
Thread 5921
Thread 5922
Thread 5923
Thread 5924
Thread 5925
Thread 5926
Thread 5927
Thread 5928
Thread 5929
Thread 5930
Thread 5931
Thread 5932
Thread 5933
Thread 5934
Thread 5935
Thread 5936
Thread 5937
Thread 5938
Thread 5939
Thread 5940
Thread 5941
Thread 5942
Thread 5943
Thread 5944
Thread 5945
Thread 5946
Thread 5947
Thread 5948
Thread 5949
Thread 5950
Thread 5951
Thread 5952
Thread 5953
Thread 5954
Thread 5955
Thread 5956
Thread 5957
Thread 5958
Thread 5959
Thread 5960
Thread 5961
Thread 5962
Thread 5963
Thread 5964
Thread 5965
Thread 5966
Thread 5967
Thread 5968
Thread 5969
Thread 5970
Thread 5971
Thread 5972
Thread 5973
Thread 5974
Thread 5975
Thread 5976
Thread 5977
Thread 5978
Thread 5979
Thread 5980
Thread 5981
Thread 5982
Thread 5983
Thread 5984
Thread 5985
Thread 5986
Thread 5987
Thread 5988
Thread 5989
Thread 5990
Thread 5991
Thread 5992
Thread 5993
Thre

Thread 8823
Thread 8824
Thread 8825
Thread 8826
Thread 8827
Thread 8828
Thread 8829
Thread 8830
Thread 8831
Thread 8832
Thread 8833
Thread 8834
Thread 8835
Thread 8836
Thread 8837
Thread 8838
Thread 8839
Thread 8840
Thread 8841
Thread 8842
Thread 8843
Thread 8844
Thread 8845
Thread 8846
Thread 8847
Thread 8848
Thread 8849
Thread 8850
Thread 8851
Thread 8852
Thread 8853
Thread 8854
Thread 8855
Thread 8856
Thread 8857
Thread 8858
Thread 8859
Thread 8860
Thread 8861
Thread 8862
Thread 8863
Thread 8864
Thread 8865
Thread 8866
Thread 8867
Thread 8868
Thread 8869
Thread 8870
Thread 8871
Thread 8872
Thread 8873
Thread 8874
Thread 8875
Thread 8876
Thread 8877
Thread 8878
Thread 8879
Thread 8880
Thread 8881
Thread 8882
Thread 8883
Thread 8884
Thread 8885
Thread 8886
Thread 8887
Thread 8888
Thread 8889
Thread 8890
Thread 8891
Thread 8892
Thread 8893
Thread 8894
Thread 8895
Thread 8896
Thread 8897
Thread 8898
Thread 8899
Thread 8900
Thread 8901
Thread 8902
Thread 8903
Thread 8904
Thread 8905
Thre

Thread 11910
Thread 11911
Thread 11912
Thread 11913
Thread 11914
Thread 11915
Thread 11916
Thread 11917
Thread 11918
Thread 11919
Thread 11920
Thread 11921
Thread 11922
Thread 11923
Thread 11924
Thread 11925
Thread 11926
Thread 11927
Thread 11928
Thread 11929
Thread 11930
Thread 11931
Thread 11932
Thread 11933
Thread 11934
Thread 11935
Thread 11936
Thread 11937
Thread 11938
Thread 11939
Thread 11940
Thread 11941
Thread 11942
Thread 11943
Thread 11944
Thread 11945
Thread 11946
Thread 11947
Thread 11948
Thread 11949
Thread 11950
Thread 11951
Thread 11952
Thread 11953
Thread 11954
Thread 11955
Thread 11956
Thread 11957
Thread 11958
Thread 11959
Thread 11960
Thread 11961
Thread 11962
Thread 11963
Thread 11964
Thread 11965
Thread 11966
Thread 11967
Thread 11968
Thread 11969
Thread 11970
Thread 11971
Thread 11972
Thread 11973
Thread 11974
Thread 11975
Thread 11976
Thread 11977
Thread 11978
Thread 11979
Thread 11980
Thread 11981
Thread 11982
Thread 11983
Thread 11984
Thread 11985
Thread 11986

...

Main    : before joining thread 8056.
Main    : before joining thread 8057.
Main    : before joining thread 8058.
Main    : before joining thread 8059.
Main    : before joining thread 8060.
Main    : before joining thread 8061.
Main    : before joining thread 8062.
Main    : before joining thread 8063.
Main    : before joining thread 8064.
Main    : before joining thread 8065.
Main    : before joining thread 8066.
Main    : before joining thread 8067.
Main    : before joining thread 8068.
Main    : before joining thread 8069.
Main    : before joining thread 8070.
Main    : before joining thread 8071.
Main    : before joining thread 8072.
Main    : before joining thread 8073.
Main    : before joining thread 8074.
Main    : before joining thread 8075.
Main    : before joining thread 8076.
Main    : before joining thread 8077.
Main    : before joining thread 8078.
Main    : before joining thread 8079.
Main    : before joining thread 8080.
Main    : before joining thread 8081.
Main    : be

Main    : before joining thread 11056.
Main    : before joining thread 11057.
Main    : before joining thread 11058.
Main    : before joining thread 11059.
Main    : before joining thread 11060.
Main    : before joining thread 11061.
Main    : before joining thread 11062.
Main    : before joining thread 11063.
Main    : before joining thread 11064.
Main    : before joining thread 11065.
Main    : before joining thread 11066.
Main    : before joining thread 11067.
Main    : before joining thread 11068.
Main    : before joining thread 11069.
Main    : before joining thread 11070.
Main    : before joining thread 11071.
Main    : before joining thread 11072.
Main    : before joining thread 11073.
Main    : before joining thread 11074.
Main    : before joining thread 11075.
Main    : before joining thread 11076.
Main    : before joining thread 11077.
Main    : before joining thread 11078.
Main    : before joining thread 11079.
Main    : before joining thread 11080.
Main    : before joining 

Main    : before joining thread 14227.
Main    : before joining thread 14228.
Main    : before joining thread 14229.
Main    : before joining thread 14230.
Main    : before joining thread 14231.
Main    : before joining thread 14232.
Main    : before joining thread 14233.
Main    : before joining thread 14234.
Main    : before joining thread 14235.
Main    : before joining thread 14236.
Main    : before joining thread 14237.
Main    : before joining thread 14238.
Main    : before joining thread 14239.
Main    : before joining thread 14240.
Main    : before joining thread 14241.
Main    : before joining thread 14242.
Main    : before joining thread 14243.
Main    : before joining thread 14244.
Main    : before joining thread 14245.
Main    : before joining thread 14246.
Main    : before joining thread 14247.
Main    : before joining thread 14248.
Main    : before joining thread 14249.
Main    : before joining thread 14250.
Main    : before joining thread 14251.
Main    : before joining 

Main    : before joining thread 17476.
Main    : before joining thread 17477.
Main    : before joining thread 17478.
Main    : before joining thread 17479.
Main    : before joining thread 17480.
Main    : before joining thread 17481.
Main    : before joining thread 17482.
Main    : before joining thread 17483.
Main    : before joining thread 17484.
Main    : before joining thread 17485.
Main    : before joining thread 17486.
Main    : before joining thread 17487.
Main    : before joining thread 17488.
Main    : before joining thread 17489.
Main    : before joining thread 17490.
Main    : before joining thread 17491.
Main    : before joining thread 17492.
Main    : before joining thread 17493.
Main    : before joining thread 17494.
Main    : before joining thread 17495.
Main    : before joining thread 17496.
Main    : before joining thread 17497.
Main    : before joining thread 17498.
Main    : before joining thread 17499.
Main    : before joining thread 17500.
Main    : before joining 

Main    : before joining thread 20554.
Main    : before joining thread 20555.
Main    : before joining thread 20556.
Main    : before joining thread 20557.
Main    : before joining thread 20558.
Main    : before joining thread 20559.
Main    : before joining thread 20560.
Main    : before joining thread 20561.
Main    : before joining thread 20562.
Main    : before joining thread 20563.
Main    : before joining thread 20564.
Main    : before joining thread 20565.
Main    : before joining thread 20566.
Main    : before joining thread 20567.
Main    : before joining thread 20568.
Main    : before joining thread 20569.
Main    : before joining thread 20570.
Main    : before joining thread 20571.
Main    : before joining thread 20572.
Main    : before joining thread 20573.
Main    : before joining thread 20574.
Main    : before joining thread 20575.
Main    : before joining thread 20576.
Main    : before joining thread 20577.
Main    : before joining thread 20578.
Main    : before joining 

In [27]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = var.labels,
      color = var.color
    ),
    link = dict(
      source = var.source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = var.target,
      value = var.values
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
# fig.show()

In [28]:
fig.write_html("diagram.html")